In [5]:
import os
import logging
import time
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

def image_download(driver, keyword, image_cnt):
    # 이미지 저장 경로
    logging.info(keyword + ' Download Start')
    save_dir = os.path.join(os.getcwd(), "flower_img", keyword)
    os.makedirs(save_dir, exist_ok=True)

    download_cnt = 0
    downloaded_images = set()  # 이미 다운로드한 이미지 URL을 저장하는 집합

    while download_cnt < image_cnt:
        time.sleep(3)
        # 페이지 맨 아래로 스크롤
        driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
        logging.info('scroll down')

        time.sleep(3)
        try:
            # '더보기' 버튼이 보이면 클릭
            load_more_button = driver.find_element(By.XPATH, '//*[@id="islmp"]/div/div/div/div/div[1]/div[2]/div[2]/input')
            if load_more_button.is_displayed():
                load_more_button.click()
                time.sleep(3)
                logging.info('button click')
        except:
            pass
        
        time.sleep(3)
        try:
            # 이미지 없을시 break
            no_more_content = driver.find_element(By.XPATH, '//div[@class="K25wae"]//*[text()="더 이상 표시할 콘텐츠가 없습니다."]')
            if no_more_content.is_displayed():
                logging.info('content end')
                break
        except:
            pass

        # 이미지 정보 추출
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        image_info_list = soup.find_all('img', class_='rg_i')

        # 이미지 다운로드
        for image_info in image_info_list:
            if download_cnt >= image_cnt:
                break
            if 'src' in image_info.attrs:
                image_url = image_info['src']
                if image_url not in downloaded_images:  # 이미지 중복 체크
                    downloaded_images.add(image_url)
                    image_filename = f"{keyword.replace(' ', '_')}_{download_cnt}.jpg"
                    image_path = os.path.join(save_dir, image_filename)
                    urllib.request.urlretrieve(image_url, image_path)
                    logging.info(f'{image_filename} image download')
                    download_cnt += 1

    logging.info(keyword + ' Download End')


In [6]:
# 드라이버 초기화 및 페이지 열기
keyword = 'cherry blossom korea'
driver = webdriver.Chrome()  # Chrome 웹 드라이버 실행
URL = 'https://www.google.com/search?tbm=isch&q='
driver.get(URL + keyword)  # 검색어를 포함한 URL로 이동
image_cnt = 300


# 이미지 다운로드 함수 호출
image_download(driver, keyword, image_cnt)
driver.close()